# Performer: Fake News

## Setup

In [18]:
import trax
import numpy as np
import tqdm
import spacy

2022-08-31 14:59:48.061045: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-31 14:59:48.061117: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-31 14:59:48.061144: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-31 14:59:48.063519: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-08-31 14:59:48.063551: W tensorflow/stream_executor/platform/default/dso_loader

In [16]:
trax.fastmath.set_backend('jax')

In [6]:
!nvidia-smi

Wed Aug 31 14:34:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.57       Driver Version: 516.59       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 12%   55C    P0    49W / 215W |   2247MiB /  8192MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Dataset

The dataset we are using may be found here: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset

In [3]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 98%|█████████████████████████████████████ | 40.0M/41.0M [00:01<00:00, 32.9MB/s]
100%|██████████████████████████████████████| 41.0M/41.0M [00:01<00:00, 30.6MB/s]


In [10]:
!unzip fake-and-real-news-dataset.zip

Archive:  fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [11]:
!ls

 Fake.csv   'download?datasetVersionNumber=1'   requirements.txt
 README.md   fake-and-real-news-dataset.zip
 True.csv    fakenews.ipynb


## Model

### Tokenization

In [51]:
sentence = 'Andrew had a migraine yesterday'

In [49]:
!wget https://raw.githubusercontent.com/google/trax/master/trax/data/testdata/en_8k.subword

--2022-08-31 15:15:01--  https://raw.githubusercontent.com/google/trax/master/trax/data/testdata/en_8k.subword
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59307 (58K) [text/plain]
Saving to: ‘en_8k.subword’

en_8k.subword       100%[===================>]  57.92K  --.-KB/s    in 0.01s   

2022-08-31 15:15:01 (3.90 MB/s) - ‘en_8k.subword’ saved [59307/59307]



In [50]:
tokenized = list(trax.data.tokenize(iter([sentence]),
                                    vocab_dir='.',
                                    vocab_file='en_8k.subword'))[0]

In [52]:
tokenized

array([2379,  103,   12, 6170, 4012,   22, 1180, 1213,  288])

### Model